## Fundamentus

This script reads all stocks data available in https://fundamentus.com.br/ using scrapers.\
Giving you the possibility to filter out only the relevant stocks as desired.

In [ ]:
import os
import numpy
import pandas
import sys

from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.fundamentus import (
    StockListScraper,
    StockDetailsScraper,
)

### Reading List of Stocks (plus general stock data)

In [2]:
stocks_general_data_df = StockListScraper().dataframe_format

# show most liquid stocks (it also shows only the first X columns)
stocks_general_data_df.sort_values("Liq.2meses", ascending=False, inplace=True)
print(stocks_general_data_df[stocks_general_data_df.columns[0:12]][0:5].to_string(index=False))
print(f"--- Total stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

Papel  Cotação   P/L  P/VP   PSR Div.Yield  P/Ativo  P/Cap.Giro  P/EBIT  P/Ativ Circ.Liq  EV/EBIT  EV/EBITDA
VALE3    68.36  6.53  1.63 1.504     8.89%    0.696       60.23    3.88            -1.78     4.51       3.76
PETR4    39.28  3.74  1.33 0.955    18.45%    0.500      -70.58    2.13            -1.04     3.12       2.38
ITUB4    32.33 10.20  1.81 0.000     3.89%    0.000        0.00    0.00             0.00     0.00       0.00
BBDC4    15.36 11.00  1.02 0.000     7.25%    0.000        0.00    0.00             0.00     0.00       0.00
BBAS3    56.58  4.85  1.01 0.000     8.07%    0.000        0.00    0.00             0.00     0.00       0.00
--- Total stock tickers in the list: 983 ---


### Filter out stocks


In [3]:
# by trading liquidity
stocks_general_data_df = stocks_general_data_df[stocks_general_data_df["Liq.2meses"] >= 1_000_000]

# show stock tickers count after filter applied
print(f"--- Filtered stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

--- Filtered stock tickers in the list: 237 ---


### Reading stocks detail data (one by one)

In [ ]:
stocks_full_data_dict = {}

for ticker in tqdm(stocks_general_data_df.index):
    stocks_full_data_dict[ticker] = StockDetailsScraper(ticker).dict_format

stocks_full_data_df = pandas.DataFrame.from_dict(stocks_full_data_dict, orient="index")

### Check stock liquidity (checking quote price date column)

In [5]:
stocks_full_data_df["Data últ cot"] = pandas.to_datetime(stocks_full_data_df["Data últ cot"], format="%d/%m/%Y")
stocks_full_data_df = stocks_full_data_df[stocks_full_data_df["Data últ cot"] == stocks_full_data_df["Data últ cot"].max()]

# show stock tickers count after filter applied
print(f"--- Filtered stocks data in the list: {stocks_full_data_df.shape[0]} ---")

--- Filtered stocks data in the list: 235 ---


### Filter out relevant columns

In [6]:
cols = [
    "Papel", "Tipo", "Empresa", "Setor", "Subsetor",
    "Cotação", "Min 52 sem", "Max 52 sem", "Vol $ méd (2m)",
    "Nro. Ações", "Valor de mercado", "Valor da firma",
    "P/L", "LPA", "P/VP", "VPA", "P/EBIT", "Marg. Bruta",
    "PSR", "Marg. EBIT", "P/Ativos", "Marg. Líquida", 
    "P/Cap. Giro", "EBIT / Ativo", "P/Ativ Circ Liq",
    "Div. Yield", "ROIC",  "ROE", "EV / EBITDA", "Liquidez Corr",
    "EV / EBIT", "Div Br/ Patrim", "Cres. Rec (5a)",
    "Giro Ativos", "Ativo", "Dív. Bruta", "Disponibilidades",
    "Dív. Líquida", "Ativo Circulante", "Patrim. Líq",
    "Receita Líquida", "EBIT", "Lucro Líquido",  # <- Last 12 months infos
    "Receita Líquida_", "EBIT_",  "Lucro Líquido_",  # <- Last quarted infos
    "Depósitos", "Cart. de Crédito", "Result Int Financ",  # <- Last 12 months infos
    "Result Int Financ_", "Rec Serviços", "Rec Serviços_",  # <- Last quarted infos
]
stocks_full_data_df = stocks_full_data_df[cols]

# show most liquid stocks data (it also shows only the first X columns)
print(stocks_full_data_df[stocks_full_data_df.columns[0:6]][0:5].to_string(index=False))
print(f"--- Total stock data rows in the list: {stocks_full_data_df.shape[0]} ---")

Papel  Tipo                   Empresa                           Setor                          Subsetor  Cotação
VALE3 ON NM                VALE ON NM                       Mineração                Minerais Metálicos    68.36
PETR4    PN              PETROBRAS PN Petróleo, Gás e Biocombustíveis Exploração, Refino e Distribuição    39.28
ITUB4 PN N1        ITAUUNIBANCO PN N1      Intermediários Financeiros                            Bancos    32.33
BBDC4 PN N1 BANCO BRADESCO S.A. PN N1      Intermediários Financeiros                            Bancos    15.36
BBAS3    ON   BANCO DO BRASIL S.A. ON      Intermediários Financeiros                            Bancos    56.58
--- Total stock data rows in the list: 235 ---


### Clean up "-" values from cells, replacing them with empty values

In [8]:
stocks_full_data_df = stocks_full_data_df.replace('-', numpy.nan)

### Output result into a file

In [9]:
stocks_full_data_df.to_csv("../outputs/fundamentus.csv", index=False)